# Point to grid conversion

This code takes the georeferenced points and related them to a fishnet grid of 1km x 1km scale. 

This will be for both for Australian Plague Locust and Desert Locust




In [1]:
import pandas as pd
import geopandas as gpd
import os

In [3]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.getcwd()

'/home/datascience/herbivore_nutrient_interactions'

In [6]:
# This function takes the point dataset and creates a sparse fishnet grid that overlaps with all points. 
# there is a user species cell size and few other handy parameters
# it defaults to parallel processing but can be turned off

# it creates a bounding box, creates a coarse grid, filters out unused grids and then within each coarse grid, it creates the fine scale grid
# at the speicifed cell size and then removes unused grids.

with open('scripts/functions/fishnet_grid_function.py') as f:
    code = f.read()

# Execute the code
exec(code)


## Australian plague locust data

This dataset is comprised of several species but the records are biased towards the more economically important species (e.g. australian plauge locust)

## Species identifier

The `Species` column is the identifier per species

 - 10 = no locust/grasshopper found
 - 11 = Australian plague locust (*Chortocietes terminifera*)
 - 12 = Spur-throated locust (*Austracris guttulosa*)
 - 15 = Small plague locust (*Austroicetes cruciata*)
 - 19 = Eastern plague grasshopper (*Oedaleus australis*)

### Species data management

For each species, we need to combined the species specific ID with the `10` observations (nil-observations) and observations in the other species categories that are nil. This will provide us with all the records that a specific grasshopper species was present and combine it with all the nil observation possible.



In [8]:
df = pd.read_csv('data/raw/survey_data/APLC_surveys_2000-2017.csv')

In [9]:
import pandas as pd

def filter_and_update_species(df, species_list, target_species):
    and_nil = df['Species'].isin(species_list)
    all_other_nil = (
        ~df['Species'].isin(species_list) &
        (df['Nymph Density'] == 0) &
        (df['Adult Density'] == 0)
    )
    filtered_df = df[and_nil | all_other_nil].copy()  # Use copy() to avoid SettingWithCopyWarning
    filtered_df['Species'] = target_species
    return filtered_df

# Define the list of target species and corresponding species lists
species_combinations = [
    ([10, 11], 11),
    ([10, 12], 12),
    ([10, 15], 15),
    ([10, 19], 19)
]

# Use a list comprehension to generate all DataFrames in one step
full_species_data = pd.concat(
    [filter_and_update_species(df, species_list, target_species) 
     for species_list, target_species in species_combinations], 
    ignore_index=True
).drop_duplicates()  # Optionally drop duplicates

# final_df now contains the combined result
full_species_data.head()

,Source,Date,Longitude,Latitude,Longitude2,Latitude2,Species,Data Quality,Nymph Stage,Nymph Density,Adult Stage,Adult Density
0,1,7-Apr-17,143.144968,-24.760067,143.144968,-24.760067,11,6,0,0,7,1
1,1,7-Apr-17,143.276912,-24.473368,143.276912,-24.473368,11,6,0,0,0,0
2,1,7-Apr-17,143.278365,-24.643177,143.278365,-24.643177,11,6,0,0,7,1
3,1,7-Apr-17,143.367765,-24.354893,143.367765,-24.354893,11,6,0,0,7,1
4,1,7-Apr-17,143.459170,-24.319528,143.459170,-24.319528,11,6,0,0,7,1


## Fishnet grid construction

I first create the sparse fishnet grid (roughly 1km x 1km) and then aggregate the species data to the polygons. This is done BY species and not between.

In [13]:

aplc_survey_dataframe = create_grids_parallel(
        full_species_data,
        longitude_column='Longitude',
        latitude_column='Latitude',
        csv_crs='EPSG:4326',
        grid_crs='EPSG:4326',
        transformation_crs='EPSG:4326',
        final_cell_size=0.01,
        coarse_cell_size=None,
        parallel=True
    )



Processing Polygons: 100%|██████████| 1315/1315 [00:13<00:00, 94.54it/s]


In [16]:
# Add an index to the polygons
## this will allow us to join tables together (and make GEE easier)

aplc_survey_dataframe = gpd.GeoDataFrame(aplc_survey_dataframe)
aplc_survey_dataframe['polygon_id'] = aplc_survey_dataframe.index
aplc_survey_dataframe.set_crs(epsg=4326, inplace=True)

,geometry,polygon_id
13,"POLYGON ((131.66364 -28.11974, 131.66364 -28.1...",13
3072,"POLYGON ((133.20694 -26.35200, 133.20694 -26.3...",3072
4563,"POLYGON ((133.18694 -26.24757, 133.18694 -26.2...",4563
4620,"POLYGON ((133.19694 -26.07757, 133.19694 -26.0...",4620
4626,"POLYGON ((133.19694 -26.01757, 133.19694 -26.0...",4626
...,...,...
2101240,"POLYGON ((151.10087 -27.06644, 151.10087 -27.0...",2101240
2101580,"POLYGON ((151.18087 -26.86644, 151.18087 -26.8...",2101580
2101656,"POLYGON ((151.20087 -26.90644, 151.20087 -26.8...",2101656
2102441,"POLYGON ((151.00087 -26.66200, 151.00087 -26.6...",2102441


### Lets summarize the APL point data to the polygon grid data

In [17]:
columns_to_keep = ['Longitude', 'Latitude', 'Date','Species','Data Quality','Nymph Density','Adult Density']
APL_dat = full_species_data[columns_to_keep]


In [18]:
# Convert DataFrame to GeoDataFrame

gdf_points = gpd.GeoDataFrame(
    APL_dat,
    geometry=gpd.points_from_xy(APL_dat.Longitude, APL_dat.Latitude)
)

gdf_points.set_crs(epsg=4326, inplace=True)


,Longitude,Latitude,Date,Species,Data Quality,Nymph Density,Adult Density,geometry
0,143.144968,-24.760067,7-Apr-17,11,6,0,1,POINT (143.14497 -24.76007)
1,143.276912,-24.473368,7-Apr-17,11,6,0,0,POINT (143.27691 -24.47337)
2,143.278365,-24.643177,7-Apr-17,11,6,0,1,POINT (143.27837 -24.64318)
3,143.367765,-24.354893,7-Apr-17,11,6,0,1,POINT (143.36776 -24.35489)
4,143.459170,-24.319528,7-Apr-17,11,6,0,1,POINT (143.45917 -24.31953)
...,...,...,...,...,...,...,...,...
316058,143.073900,-25.521900,5-Jan-00,19,3,0,0,POINT (143.07390 -25.52190)
316059,143.307100,-25.209900,5-Jan-00,19,3,0,0,POINT (143.30710 -25.20990)
316060,143.839400,-25.032800,5-Jan-00,19,3,0,0,POINT (143.83940 -25.03280)
316061,143.921900,-24.971200,5-Jan-00,19,3,0,0,POINT (143.92190 -24.97120)


In [19]:
joined_gdf = gpd.sjoin(gdf_points, aplc_survey_dataframe, how="left", predicate='within')


In [20]:
# I wanted to ensure every point has an assoicated polygon_id 

## which they do

na_counts = joined_gdf.isna().sum()

print(na_counts)

Longitude        0
Latitude         0
Date             0
Species          0
Data Quality     0
Nymph Density    0
Adult Density    0
geometry         0
index_right      0
polygon_id       0
dtype: int64


In [21]:

# Custom aggregation function to get min, median, and max dates
def date_aggregations(series):
    series = pd.to_datetime(series, format='%d-%b-%y') # converts to a datatime
    return [series.min(), series.median(), series.max()]

# aggregation function to count unique values
def count_unique_values(series):
    return series.value_counts().to_dict()

# Group by polygon geometry and aggregate the data
aggregated_gdf = joined_gdf.groupby(['Species', 'polygon_id']).agg({
    'Date': date_aggregations,
    'Longitude': 'median',
    'Latitude': 'median',
    'Data Quality': count_unique_values,
    'Nymph Density': count_unique_values,
    'Adult Density': count_unique_values
}).reset_index()


In [26]:
aggregated_gdf

,Species,polygon_id,Date,Longitude,Latitude,Data Quality,Nymph Density,Adult Density
0,11,13,"[2014-03-08 00:00:00, 2014-03-08 00:00:00, 201...",131.655639,-28.119211,{6: 1},{0: 1},{0: 1}
1,11,3072,"[2017-04-01 00:00:00, 2017-04-01 00:00:00, 201...",133.206243,-26.350383,{6: 1},{0: 1},{0: 1}
2,11,4563,"[2006-03-16 00:00:00, 2006-03-16 00:00:00, 200...",133.186734,-26.237582,{6: 1},{0: 1},{0: 1}
3,11,4620,"[2006-03-16 00:00:00, 2006-03-16 00:00:00, 200...",133.195693,-26.075735,{6: 1},{0: 1},{0: 1}
4,11,4626,"[2017-04-01 00:00:00, 2017-04-01 00:00:00, 201...",133.194124,-26.009872,{6: 1},{0: 1},{0: 1}
...,...,...,...,...,...,...,...,...
177608,19,2101123,"[2000-12-16 00:00:00, 2000-12-16 00:00:00, 200...",151.065400,-27.032500,{3: 1},{0: 1},{0: 1}
177609,19,2101201,"[2012-03-11 00:00:00, 2012-03-11 00:00:00, 201...",151.082595,-27.048465,{6: 1},{0: 1},{0: 1}
177610,19,2101240,"[2011-11-12 00:00:00, 2011-11-12 00:00:00, 201...",151.098285,-27.062210,{6: 1},{0: 1},{0: 1}
177611,19,2101580,"[2011-02-04 00:00:00, 2011-02-04 00:00:00, 201...",151.175738,-26.865963,{6: 1},{0: 1},{0: 1}


In [27]:
# Lets count up the total number of observations in the dataframe:

# Define the aggregation functions for each column
agg_funcs = {
    'Data Quality': 'count',  
    'Adult Density': 'count',    
    'Nymph Density': 'count'    
}

# Perform groupby operation and aggregate
grouped_joined_gdf = joined_gdf.groupby(['Species', 'polygon_id']).agg(agg_funcs).reset_index()

# Rename the aggregated columns
grouped_joined_gdf.rename(columns={
    'Data Quality': 'Data Quality Total Count',
    'Adult Density': 'Adult Density Total Count',
    'Nymph Density': 'Nymph Density Total Count'
}, inplace=True)


# These three columns should agree with one another....
## if any rows dont -- this command filters for them
grouped_joined_gdf[
    (grouped_joined_gdf['Data Quality Total Count'] != grouped_joined_gdf['Adult Density Total Count']) |
    (grouped_joined_gdf['Data Quality Total Count'] != grouped_joined_gdf['Nymph Density Total Count']) |
    (grouped_joined_gdf['Adult Density Total Count'] != grouped_joined_gdf['Nymph Density Total Count'])
]

,Species,polygon_id,Data Quality Total Count,Adult Density Total Count,Nymph Density Total Count


In [28]:
final_df = pd.merge(aggregated_gdf, grouped_joined_gdf, on=['Species', 'polygon_id'], how='left')
final_df = pd.merge(final_df, aplc_survey_dataframe, on='polygon_id', how='left')
final_df.head()
na_counts = final_df.isna().sum()

print(na_counts)

Species                      0
polygon_id                   0
Date                         0
Longitude                    0
Latitude                     0
Data Quality                 0
Nymph Density                0
Adult Density                0
Data Quality Total Count     0
Adult Density Total Count    0
Nymph Density Total Count    0
geometry                     0
dtype: int64


### The APLC data is complete -- lets write to disk 

In [29]:
final_df.to_csv('data/processed/spatial_modeling/aplc_data_aggregated_to_polygon_grid.csv')